In [1]:
from carpool_data import posts, derived_posts

In [2]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import matplotlib as mpl
import string
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

In [3]:
#vectorizing the data (count vectorization)
def split_by_comma(x):
    return x.split(",")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

count_vect = CountVectorizer(analyzer = split_by_comma)

#the training set of posts for number of routes
#in particular, the classified posts for number of routes
posts_tr_dos = posts[~posts["driving_or_searching"].isnull()].reset_index(drop = True)

X_count = count_vect.fit_transform(posts_tr_dos["tockenized_message"])
X_count_feat = pd.DataFrame(X_count.toarray())
Y = posts_tr_dos["driving_or_searching"]

In [4]:
#vectorizing the data (tfidf)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = split_by_comma)
X_tfidf = tfidf_vect.fit_transform(posts_tr_dos["tockenized_message"])
X_tfidf_feat= pd.DataFrame(X_tfidf.toarray())

In [5]:
#vectorizing the data (bigram)
bigram_vect = CountVectorizer(ngram_range = (2,2), analyzer = split_by_comma)
X_bigram = bigram_vect.fit_transform(posts_tr_dos["tockenized_message"])
X_bigram_feat= pd.DataFrame(X_bigram.toarray())

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold,cross_val_score

rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300], 'max_depth': [60,90,None]}

We will now compare the count vectorizer vs. tfidf vs. n-grams across different parameter settings, using Grid Search

In [7]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(rf, param, cv = 5, n_jobs = -1, return_train_score= True)
gs_fit = gs.fit(X_count_feat, Y)
pd.DataFrame(gs_fit.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.041682,0.002812,0.964129,0.998901,60,10,"{'max_depth': 60, 'n_estimators': 10}",7,0.930657,0.998168,...,0.970803,0.999084,0.974265,1.000000,0.977941,0.999086,0.003140,0.000744,0.017145,0.000685
1,0.845442,0.035507,0.975842,1.000000,60,150,"{'max_depth': 60, 'n_estimators': 150}",4,0.970803,1.000000,...,0.970803,1.000000,0.981618,1.000000,0.985294,1.000000,0.306896,0.015005,0.006302,0.000000
2,2.469171,0.097460,0.978038,1.000000,60,300,"{'max_depth': 60, 'n_estimators': 300}",2,0.970803,1.000000,...,0.978102,1.000000,0.985294,1.000000,0.985294,1.000000,0.024369,0.005825,0.006478,0.000000
3,0.097257,0.005818,0.964129,0.997987,90,10,"{'max_depth': 90, 'n_estimators': 10}",7,0.959854,0.996337,...,0.967153,0.997253,0.981618,0.997258,0.959559,0.999086,0.011057,0.000751,0.009871,0.001346
4,1.252332,0.041310,0.974378,1.000000,90,150,"{'max_depth': 90, 'n_estimators': 150}",6,0.959854,1.000000,...,0.974453,1.000000,0.988971,1.000000,0.981618,1.000000,0.150028,0.008181,0.010277,0.000000
5,2.685344,0.071592,0.978770,1.000000,90,300,"{'max_depth': 90, 'n_estimators': 300}",1,0.970803,1.000000,...,0.970803,1.000000,0.992647,1.000000,0.985294,1.000000,0.136203,0.009224,0.008719,0.000000
6,0.099868,0.004814,0.956076,0.998719,None,10,"{'max_depth': None, 'n_estimators': 10}",9,0.937956,0.998168,...,0.967153,1.000000,0.963235,0.998172,0.955882,0.998172,0.014153,0.000750,0.010034,0.000732
7,1.229869,0.045723,0.975842,1.000000,None,150,"{'max_depth': None, 'n_estimators': 150}",4,0.967153,1.000000,...,0.970803,1.000000,0.985294,1.000000,0.981618,1.000000,0.175912,0.016493,0.006712,0.000000
8,1.796176,0.042924,0.976574,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",3,0.967153,1.000000,...,0.974453,1.000000,0.985294,1.000000,0.981618,1.000000,0.029801,0.015233,0.006310,0.000000


In [8]:
gs_fit = gs.fit(X_tfidf, Y)
pd.DataFrame(gs_fit.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.040710,0.002214,0.965593,0.998535,60,10,"{'max_depth': 60, 'n_estimators': 10}",7,0.941606,0.998168,...,0.963504,0.998168,0.981618,1.000000,0.974265,0.998172,0.004170,3.977049e-04,0.013523,0.000732
1,0.783169,0.023362,0.978770,1.000000,60,150,"{'max_depth': 60, 'n_estimators': 150}",1,0.970803,1.000000,...,0.970803,1.000000,0.988971,1.000000,0.985294,1.000000,0.186988,4.815006e-03,0.007399,0.000000
2,1.942165,0.064873,0.972914,1.000000,60,300,"{'max_depth': 60, 'n_estimators': 300}",4,0.956204,1.000000,...,0.970803,1.000000,0.985294,1.000000,0.974265,1.000000,0.064836,6.079339e-03,0.009653,0.000000
3,0.065975,0.003008,0.964129,0.998536,90,10,"{'max_depth': 90, 'n_estimators': 10}",8,0.930657,0.999084,...,0.974453,0.999084,0.966912,0.998172,0.981618,0.999086,0.005783,4.672031e-07,0.017618,0.000733
4,1.024624,0.029479,0.972182,1.000000,90,150,"{'max_depth': 90, 'n_estimators': 150}",5,0.967153,1.000000,...,0.970803,1.000000,0.974265,1.000000,0.977941,1.000000,0.091411,2.825166e-03,0.003647,0.000000
5,1.920508,0.052941,0.978770,1.000000,90,300,"{'max_depth': 90, 'n_estimators': 300}",1,0.959854,1.000000,...,0.978102,1.000000,0.992647,1.000000,0.985294,1.000000,0.053939,1.507569e-03,0.010898,0.000000
6,0.072392,0.004011,0.960469,0.998718,None,10,"{'max_depth': None, 'n_estimators': 10}",9,0.934307,1.000000,...,0.967153,0.997253,0.981618,0.999086,0.959559,1.000000,0.011845,1.520678e-03,0.015348,0.001242
7,0.950929,0.031584,0.971449,1.000000,None,150,"{'max_depth': None, 'n_estimators': 150}",6,0.945255,1.000000,...,0.974453,1.000000,0.977941,1.000000,0.985294,1.000000,0.030157,2.496129e-03,0.013704,0.000000
8,1.781438,0.052038,0.973646,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",3,0.956204,1.000000,...,0.974453,1.000000,0.988971,1.000000,0.974265,1.000000,0.028396,2.402144e-03,0.010398,0.000000


In [9]:
gs_fit = gs.fit(X_bigram, Y)
pd.DataFrame(gs_fit.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.034605,0.001789,0.969253,0.997804,60,10,"{'max_depth': 60, 'n_estimators': 10}",7,0.963504,0.998168,...,0.963504,0.998168,0.974265,0.996344,0.985294,0.998172,0.002549,0.000393,0.009339,0.000730
1,0.749794,0.022460,0.973646,1.000000,60,150,"{'max_depth': 60, 'n_estimators': 150}",6,0.963504,1.000000,...,0.970803,1.000000,0.985294,1.000000,0.977941,1.000000,0.234590,0.005763,0.007387,0.000000
2,2.138086,0.069585,0.977306,1.000000,60,300,"{'max_depth': 60, 'n_estimators': 300}",2,0.967153,1.000000,...,0.974453,1.000000,0.988971,1.000000,0.977941,1.000000,0.173829,0.009614,0.007043,0.000000
3,0.066677,0.003309,0.961933,0.998353,90,10,"{'max_depth': 90, 'n_estimators': 10}",9,0.963504,1.000000,...,0.952555,0.996337,0.970588,0.997258,0.963235,1.000000,0.004060,0.000401,0.005853,0.001464
4,0.995047,0.031785,0.975842,1.000000,90,150,"{'max_depth': 90, 'n_estimators': 150}",5,0.963504,1.000000,...,0.978102,1.000000,0.985294,1.000000,0.981618,1.000000,0.100928,0.005118,0.007816,0.000000
5,2.241664,0.074497,0.976574,1.000000,90,300,"{'max_depth': 90, 'n_estimators': 300}",3,0.956204,1.000000,...,0.974453,1.000000,0.988971,1.000000,0.985294,1.000000,0.145906,0.012336,0.011417,0.000000
6,0.084222,0.003108,0.967057,0.998171,None,10,"{'max_depth': None, 'n_estimators': 10}",8,0.948905,0.999084,...,0.985401,0.999084,0.970588,0.997258,0.959559,0.997258,0.004745,0.000491,0.012251,0.000817
7,1.018007,0.031785,0.976574,1.000000,None,150,"{'max_depth': None, 'n_estimators': 150}",3,0.959854,1.000000,...,0.978102,1.000000,0.988971,1.000000,0.985294,1.000000,0.038309,0.002021,0.010439,0.000000
8,1.901156,0.050735,0.980234,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",1,0.974453,1.000000,...,0.978102,1.000000,0.988971,1.000000,0.985294,1.000000,0.024790,0.002603,0.005884,0.000000


In [10]:
#prediction function
def dos_classifier(tockenized_text):
    rf = RandomForestClassifier(n_estimators = 60, max_depth = 10, n_jobs=-1)
    rf_model = rf.fit(X_count_feat,Y)
    X = count_vect.fit(posts_tr_dos["tockenized_message"])
    message_turn_vector = X.transform([tockenized_text])
    return rf_model.predict(message_turn_vector)

In [11]:
#test
dos_classifier("anyone,going,to,york")

array(['S'], dtype=object)